In [1]:
from langchain.chat_models import init_chat_model
model = init_chat_model("groq:qwen/qwen3-32b")
response = model.invoke("What is the capital of France?")
response.content

'<think>\nOkay, so the user is asking for the capital of France. Hmm, I need to make sure I get this right. Let me think. I remember from school that France is a country in Europe. The capital... I think it\'s Paris. Wait, but wait—could I be mixing it up with another country? Let me double-check. I know that France is famous for the Eiffel Tower, which is in Paris. Also, Paris is a major city, so it\'s likely the capital. But just to be thorough, maybe I should recall other major cities in France. There\'s Lyon, Marseille, Nice, Bordeaux... none of those seem to be the capital. The capital is usually the political and administrative center, so Paris must be the right answer. I don\'t think there\'s any confusion with countries like Germany or Italy, where the capitals are Berlin and Rome respectively. Yeah, France\'s capital is definitely Paris. I can\'t think of any other city that\'s associated with the French government. The government offices, the Elysée Palace where the president

In [2]:
from langchain.tools import tool

@tool
def get_weather(location: str) -> str:
    """Get the current weather for a given location."""
    return f"The current weather in {location} is sunny with a high of 25°C."

model_with_tools = model.bind_tools([get_weather])

In [5]:
response = model_with_tools.invoke("What is the current weather in New York?")
print(response)
for tool_call in response.tool_calls:
    print(f"Tool called: {tool_call['name']}")
    print(f"Tool arguments: {tool_call['args']}")

content='' additional_kwargs={'reasoning_content': 'Okay, the user is asking for the current weather in New York. Let me check the tools available. There\'s a function called get_weather that takes a location parameter. Since the user specified New York, I need to call that function with "New York" as the location. I\'ll make sure the parameters are correctly formatted in JSON and wrap the tool call in the required XML tags.\n', 'tool_calls': [{'id': 't5z7nv593', 'function': {'arguments': '{"location":"New York"}', 'name': 'get_weather'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 103, 'prompt_tokens': 157, 'total_tokens': 260, 'completion_time': 0.149106101, 'completion_tokens_details': {'reasoning_tokens': 78}, 'prompt_time': 0.006394701, 'prompt_tokens_details': None, 'queue_time': 0.006033709, 'total_time': 0.155500802}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls

### Tool Execution Loop

In [15]:
messages = [{"role": "user", "content": "What is the current weather in New York?"}]
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    print(f"Tool called: {tool_call}")
    tool_result = get_weather.invoke(tool_call['args'])
    messages.append(tool_result)
    
final_response = model_with_tools.invoke(messages)
print(final_response.text)

Tool called: {'name': 'get_weather', 'args': {'location': 'New York'}, 'id': 'b7y2hd3m2', 'type': 'tool_call'}
The current weather in New York is sunny with a high temperature of 25°C today. It looks like a pleasant day! 😊


In [16]:
messages

[{'role': 'user', 'content': 'What is the current weather in New York?'},
 AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for the current weather in New York. Let me check the tools available. There\'s a function called get_weather that takes a location parameter. Since the user specified New York, I need to call that function with "New York" as the location. I\'ll make sure the parameters are correctly formatted in JSON. No other tools are provided, so this should be the only function call needed.\n', 'tool_calls': [{'id': 'b7y2hd3m2', 'function': {'arguments': '{"location":"New York"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 109, 'prompt_tokens': 157, 'total_tokens': 266, 'completion_time': 0.172429444, 'completion_tokens_details': {'reasoning_tokens': 84}, 'prompt_time': 0.007890518, 'prompt_tokens_details': None, 'queue_time': 0.008130026, 'total_time': 0.180319962}, 'model_name